<a href="https://colab.research.google.com/github/shinichiromizuno/QueryMultiTopic/blob/master/Multi_BERTSum_Simple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount your Google Drive.
# Connect to GPU in Google Colab.

In [ ]:
################ Preprocess ################

In [ ]:
# Configure working directories.　We use the preprocessed dataset output of Multi-BERTSum by configuring it as the source directory.
work_dir = '/content/drive/MyDrive/work_Multi_BERTSum_Simple'
source_dir = '/content/drive/MyDrive/work_Multi_BERTSum'

In [ ]:
# Install required packages and configure module path.
!pip install transformers
!pip install pyrouge
%env MODULE_PATH=/content/drive/MyDrive/QueryMultiTopic

In [ ]:
# Skipping Step1~5, as we use the preprocessed dataset output of Multi-BERTSum.

In [ ]:
# Preparing Data for Validation & Test
import json
import random
from glob import glob
import torch

src_doc_valid = []
tgt_list_valid = []
src_doc_test = []
tgt_list_test = []

for data in sorted(glob(source_dir + '/json_data_labeled/*')):
  filetype = data.split('/')[-1].split('.')[1]
  json_open = open(data)
  json_load = json.load(json_open)
  src_txt = json_load['src_txt']
  if filetype == 'valid':
    src_doc_valid.append(src_txt)
    tgt_list = []
    for i in range(1,18):
      tgt_list.append(json_load['tgt'+str(i)])
    tgt_list_valid.append(tgt_list)
  if filetype == 'test':
    src_doc_test.append(src_txt)
    tgt_list = []
    for i in range(1,18):
      tgt_list.append(json_load['tgt'+str(i)])
    tgt_list_test.append(tgt_list)

# Incorporate SDGs goal text in query text
sdgsdir = '/content/drive/MyDrive/DatasetSDGs/0-SDGs/'

all_query_txt = []
for goal in sorted(glob(sdgsdir + '/*')):
  query_txt = []
  with open(goal, mode='r') as f:
    for line in f:
      query_txt.append(line.strip())
      break
  all_query_txt.append(' '.join(query_txt))

dataset = {'src_doc_valid': src_doc_valid, 'tgt_list_valid': tgt_list_valid, 'src_doc_test': src_doc_test, 'tgt_list_test': tgt_list_test, 'all_query_txt': all_query_txt}
torch.save(dataset, work_dir + '/dataset.pt')  

In [ ]:
################ Training ################

In [ ]:
# Install Pytorch
!pip3 install torchtext==0.3.1
!pip3 install torch==1.6.0
!pip install tensorboardX

     |████████████████████████████████| 62 kB 1.2 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.11.0
    Uninstalling torchtext-0.11.0:
      Successfully uninstalled torchtext-0.11.0
     |████████████████████████████████| 748.8 MB 17 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.1+cu111 requires torch==1.10.0, but you have torch 1.6.0 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.6.0 which is incompatible.


     |████████████████████████████████| 125 kB 9.1 MB/s 


In [ ]:
# Training Models by Query
!rm -r $work_dir/models
!mkdir $work_dir/models
!mkdir logs 

for idx in range(1, 18):
  %env BERT_DATA_PATH=$source_dir/pt_data/$idx/ 
  %env MODEL_PATH=$work_dir/models/proposed$idx
  %env LOG_PATH=/content/logs/proposed$idx
  !python $MODULE_PATH/src/train.py -mode train -summarizer proposed_sigmoid -dropout 0.1 -bert_data_path $BERT_DATA_PATH -model_path $MODEL_PATH -lr 2e-3 -visible_gpus 0  -gpu_ranks 0 -world_size 1 -report_every 10 -save_checkpoint_steps 1000 -batch_size 4000 -decay_method noam -train_steps 24000 -accum_count 2 -log_file $LOG_PATH -use_interval true -warmup_steps 10000 -ff_size 2048 -inter_layers 2 -heads 8

ストリーミング出力は最後の 5000 行に切り捨てられました。
[2022-04-21 17:27:27,535 INFO] Step 21650/24000; xent: 0.05; lr: 0.0000136;  35 docs/s;   1315 sec
[2022-04-21 17:27:29,329 INFO] Loading train dataset from /content/drive/MyDrive/work_proposed/pt_data/13/train.155.Jtect.2019.pt, number of examples: 126
[2022-04-21 17:27:35,597 INFO] Step 21660/24000; xent: 0.04; lr: 0.0000136;  35 docs/s;   1323 sec
[2022-04-21 17:27:36,621 INFO] Loading train dataset from /content/drive/MyDrive/work_proposed/pt_data/13/train.119.KS.2021.pt, number of examples: 50
[2022-04-21 17:27:39,714 INFO] Loading train dataset from /content/drive/MyDrive/work_proposed/pt_data/13/train.148.GS Yuasa.2019.pt, number of examples: 154
[2022-04-21 17:27:43,572 INFO] Step 21670/24000; xent: 0.24; lr: 0.0000136;  33 docs/s;   1331 sec
[2022-04-21 17:27:48,564 INFO] Loading train dataset from /content/drive/MyDrive/work_proposed/pt_data/13/train.227.DS Pharma.2019.pt, number of examples: 205
[2022-04-21 17:27:51,690 INFO] Step 21680/24000;

In [ ]:
################ Validation ################

In [ ]:
# Validation
!cp $MODULE_PATH/bert_config_uncased_base_proposed.json ../bert_config_uncased_base.json
!rm -r $work_dir/results
!mkdir $work_dir/results
!rm -r $work_dir/logs
!mkdir $work_dir/logs
%env LOG_PATH=$work_dir/logs/proposed

for idx in range(1, 18):
  !mkdir $work_dir/results/$idx
  %env RESULT_PATH=$work_dir/results/$idx/ 
  %env BERT_DATA_PATH=$source_dir/pt_data/$idx/
  %env MODEL_PATH=$work_dir/models/proposed$idx
  !python $MODULE_PATH/src/train.py -mode valid -summarizer proposed_sigmoid -bert_data_path $BERT_DATA_PATH -visible_gpus 0  -gpu_ranks 0 -batch_size 512 -log_file $LOG_PATH  -result_path $RESULT_PATH -test_from $MODEL_PATH/model_step_5000.pt -block_trigram true
  !python $MODULE_PATH/src/train.py -mode valid -summarizer proposed_sigmoid -bert_data_path $BERT_DATA_PATH -visible_gpus 0  -gpu_ranks 0 -batch_size 512 -log_file $LOG_PATH  -result_path $RESULT_PATH -test_from $MODEL_PATH/model_step_10000.pt -block_trigram true
  !python $MODULE_PATH/src/train.py -mode valid -summarizer proposed_sigmoid -bert_data_path $BERT_DATA_PATH -visible_gpus 0  -gpu_ranks 0 -batch_size 512 -log_file $LOG_PATH  -result_path $RESULT_PATH -test_from $MODEL_PATH/model_step_15000.pt -block_trigram true
  !python $MODULE_PATH/src/train.py -mode valid -summarizer proposed_sigmoid -bert_data_path $BERT_DATA_PATH -visible_gpus 0  -gpu_ranks 0 -batch_size 512 -log_file $LOG_PATH  -result_path $RESULT_PATH -test_from $MODEL_PATH/model_step_16000.pt -block_trigram true
  !python $MODULE_PATH/src/train.py -mode valid -summarizer proposed_sigmoid -bert_data_path $BERT_DATA_PATH -visible_gpus 0  -gpu_ranks 0 -batch_size 512 -log_file $LOG_PATH  -result_path $RESULT_PATH -test_from $MODEL_PATH/model_step_17000.pt -block_trigram true
  !python $MODULE_PATH/src/train.py -mode valid -summarizer proposed_sigmoid -bert_data_path $BERT_DATA_PATH -visible_gpus 0  -gpu_ranks 0 -batch_size 512 -log_file $LOG_PATH  -result_path $RESULT_PATH -test_from $MODEL_PATH/model_step_18000.pt -block_trigram true
  !python $MODULE_PATH/src/train.py -mode valid -summarizer proposed_sigmoid -bert_data_path $BERT_DATA_PATH -visible_gpus 0  -gpu_ranks 0 -batch_size 512 -log_file $LOG_PATH  -result_path $RESULT_PATH -test_from $MODEL_PATH/model_step_19000.pt -block_trigram true
  !python $MODULE_PATH/src/train.py -mode valid -summarizer proposed_sigmoid -bert_data_path $BERT_DATA_PATH -visible_gpus 0  -gpu_ranks 0 -batch_size 512 -log_file $LOG_PATH  -result_path $RESULT_PATH -test_from $MODEL_PATH/model_step_20000.pt -block_trigram true

ストリーミング出力は最後の 5000 行に切り捨てられました。
[2022-04-16 22:04:15,150 INFO] Loading test dataset from /content/drive/MyDrive/work_proposed/pt_data/2/valid.123.KPPC.2019.pt, number of examples: 106
[2022-04-16 22:04:17,913 INFO] Loading test dataset from /content/drive/MyDrive/work_proposed/pt_data/2/valid.13.Advantest.2021.pt, number of examples: 157
[2022-04-16 22:04:21,876 INFO] Loading test dataset from /content/drive/MyDrive/work_proposed/pt_data/2/valid.134.Sapporo.2018.pt, number of examples: 240
[2022-04-16 22:04:27,709 INFO] Loading test dataset from /content/drive/MyDrive/work_proposed/pt_data/2/valid.145.Sanyo Denki.2021.pt, number of examples: 54
[2022-04-16 22:04:29,279 INFO] Loading test dataset from /content/drive/MyDrive/work_proposed/pt_data/2/valid.147.Sanwa Holdings.2021.pt, number of examples: 126
[2022-04-16 22:04:32,494 INFO] Loading test dataset from /content/drive/MyDrive/work_proposed/pt_data/2/valid.161.Sysmex.2017.pt, number of examples: 109
[2022-04-16 22:04:35,345 INFO] 

In [ ]:
# Exploring Optimal Hyper-Parameters
def predict_goals(cos_sim_list, threshold):
  src_pred_list = []
  for cos_sim in cos_sim_list:
    ones = torch.ones(cos_sim.shape).int()
    zeros = torch.zeros(cos_sim.shape).int()
    pred = torch.where(cos_sim > threshold, ones, zeros)
    src_pred_list.append(pred.tolist())
  return src_pred_list

In [ ]:
def gen_optim_list(pt_optim_dicts_all, qry, priority):
  optim_dicts_all = []
  for pt_optim_dicts in pt_optim_dicts_all:
    optim_dicts = {}
    optim_dicts[qry] = {}
    for pt_optim_dict in pt_optim_dicts[qry]:
      stpos = pt_optim_dict['stpos']
      enpos = pt_optim_dict['enpos']
      pred = pt_optim_dict['pred']
      for j in range(stpos, enpos):
        st_idx = j - stpos
        en_idx = enpos - 1 - j
        if priority == 'center':          
          pred_eval = st_idx * en_idx
        elif priority == 'top':
          pred_eval = (en_idx) ** 2
        elif priority == 'bottom':
          pred_eval = (st_idx) ** 2
        else:
          None
        if optim_dicts[qry].get(j):
          if optim_dicts[qry][j][-1] < pred_eval:
            optim_dicts[qry][j] = [pred[st_idx], pred_eval]
        else:
          optim_dicts[qry][j] = [pred[st_idx], pred_eval]
    optim_dicts_all.append(optim_dicts)
  return optim_dicts_all

In [ ]:
# Showing F1 Score and Accuracy in all and by qry
import numpy as np
from sklearn.metrics import f1_score, accuracy_score

def show_statistics_valid(all_predited_list, all_tgt_list, thres_list):
  best_all_predited_list = []
  best_results = []
  for i in range(17):
    by_thres_predicted = []
    f1_list, acc_list = [], []
    for j, thres in enumerate(thres_list):
      by_qry_predicted = []
      by_qry_tgt = []
      for doc_pred, doc_tgt in zip(all_predited_list, all_tgt_list):
        by_qry_predicted += doc_pred[j][i]
        by_qry_tgt += doc_tgt[i]
      f1 = f1_score(by_qry_predicted, by_qry_tgt)
      acc = accuracy_score(by_qry_predicted, by_qry_tgt)
      f1_list.append(f1)
      acc_list.append(acc)
    bs = np.argmax(f1_list)
    print(f'Goal{i+1}, threshold: {thres_list[bs]}, f1_score: {round(f1_list[bs], 3)}, accuracy: {round(acc_list[bs], 3)}')
    best_all_predited_list.append([doc_pred[bs][i] for doc_pred in all_predited_list])
    best_results.append((thres_list[bs], round(f1_list[bs], 3), round(acc_list[bs], 3)))
  
  best_all_predited_list_t = np.array(best_all_predited_list).T.tolist()
  single_predicted = []
  single_tgt = []
  for doc_pred, doc_tgt in zip(best_all_predited_list_t, all_tgt_list):
    for qry_pred, qry_tgt in zip(doc_pred, doc_tgt):
      single_predicted += qry_pred
      single_tgt += qry_tgt
  best_f1 = f1_score(single_predicted, single_tgt)
  best_acc = accuracy_score(single_predicted, single_tgt)
  print(f'Total Best f1_score:{round(best_f1, 3)}, accuracy:{round(best_acc, 3)}')
  return best_results

In [ ]:
import torch
from glob import glob
import numpy as np

def validate_execute(validation_list):
  best_results_list = []
  for step, align in validation_list:
    print(f'step:{step}, align:{align}')
    dataset = torch.load(glob(work_dir + '/dataset.pt')[0])
    optim_list_all = []
    for i in range(1, 18):
      pt_optim_dicts_all = torch.load(glob(work_dir + '/results/' + str(i) + '/valid.step' + str(step) + '.optim_dicts_all.pt')[0])
      optim_dicts_all = gen_optim_list(pt_optim_dicts_all, str(i), align)
      optim_list_by_qry = []
      for optim_dict in optim_dicts_all:
        optim_list_by_qry.append(torch.tensor([optim_dict[str(i)][j][0] for j in sorted(optim_dict[str(i)])]))
      optim_list_all.append(optim_list_by_qry)

    thres_range = range(0, 70, 1)
    src_pred_list = []
    for i in range(17):
      src_pred_list_by_thres = []
      for j in thres_range:
        src_pred = predict_goals(optim_list_all[i], threshold=(j/100))
        src_pred_list_by_thres.append(src_pred)
      src_pred_list.append(src_pred_list_by_thres)
    src_pred_list_t = np.array(src_pred_list).T.tolist()
    best_results = show_statistics_valid(src_pred_list_t, dataset['tgt_list_valid'], [i/100 for i in thres_range])
    best_results_list.append((step, align, best_results))
  return best_results_list

In [ ]:
validation_list = [(i, j) for j in ['center', 'top', 'bottom'] for i in range(15000, 24001, 1000)]
best_results_list = validate_execute(validation_list)
torch.save(best_results_list, work_dir + '/best_results_list.pt')  

step:15000, align:center


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


Goal1, threshold: 0.12, f1_score: 0.169, accuracy: 0.985
Goal2, threshold: 0.02, f1_score: 0.372, accuracy: 0.985
Goal3, threshold: 0.11, f1_score: 0.388, accuracy: 0.932
Goal4, threshold: 0.03, f1_score: 0.467, accuracy: 0.977
Goal5, threshold: 0.02, f1_score: 0.515, accuracy: 0.959
Goal6, threshold: 0.14, f1_score: 0.394, accuracy: 0.989
Goal7, threshold: 0.36, f1_score: 0.529, accuracy: 0.944
Goal8, threshold: 0.06, f1_score: 0.499, accuracy: 0.927
Goal9, threshold: 0.01, f1_score: 0.381, accuracy: 0.927
Goal10, threshold: 0.32, f1_score: 0.381, accuracy: 0.976
Goal11, threshold: 0.05, f1_score: 0.463, accuracy: 0.967
Goal12, threshold: 0.28, f1_score: 0.544, accuracy: 0.948
Goal13, threshold: 0.05, f1_score: 0.507, accuracy: 0.949
Goal14, threshold: 0.14, f1_score: 0.505, accuracy: 0.978
Goal15, threshold: 0.06, f1_score: 0.499, accuracy: 0.975
Goal16, threshold: 0.02, f1_score: 0.397, accuracy: 0.973
Goal17, threshold: 0.07, f1_score: 0.279, accuracy: 0.932


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


Total Best f1_score:0.451, accuracy:0.96
step:16000, align:center


...

Total Best f1_score:0.454, accuracy:0.961
step:24000, align:bottom


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


Goal1, threshold: 0.08, f1_score: 0.193, accuracy: 0.988
Goal2, threshold: 0.03, f1_score: 0.416, accuracy: 0.988
Goal3, threshold: 0.01, f1_score: 0.402, accuracy: 0.913
Goal4, threshold: 0.18, f1_score: 0.492, accuracy: 0.982
Goal5, threshold: 0.01, f1_score: 0.483, accuracy: 0.962
Goal6, threshold: 0.12, f1_score: 0.445, accuracy: 0.99
Goal7, threshold: 0.04, f1_score: 0.535, accuracy: 0.956
Goal8, threshold: 0.01, f1_score: 0.454, accuracy: 0.929
Goal9, threshold: 0.12, f1_score: 0.373, accuracy: 0.918
Goal10, threshold: 0.01, f1_score: 0.341, accuracy: 0.969
Goal11, threshold: 0.01, f1_score: 0.459, accuracy: 0.961
Goal12, threshold: 0.02, f1_score: 0.526, accuracy: 0.939
Goal13, threshold: 0.01, f1_score: 0.504, accuracy: 0.948
Goal14, threshold: 0.01, f1_score: 0.356, accuracy: 0.979
Goal15, threshold: 0.01, f1_score: 0.543, accuracy: 0.974
Goal16, threshold: 0.33, f1_score: 0.425, accuracy: 0.977
Goal17, threshold: 0.01, f1_score: 0.292, accuracy: 0.934


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


Total Best f1_score:0.44, accuracy:0.959


In [ ]:
# Showing Optimal Hyperparameters (Model, Threshold, and Alignment)
import numpy as np
best_results_list = torch.load(glob(work_dir + '/best_results_list.pt')[0])
params_list = []
for i in range(17):
  best_result = best_results_list[np.argmax([b[i][1] for steps, align, b in best_results_list])]
  print('Goal' + str(i+1), best_result[0], best_result[1], best_result[2][i][0])
  params_list.append((i+1, best_result[0], best_result[1], best_result[2][i][0]))

Goal1 17000 center 0.05
Goal2 19000 top 0.07
Goal3 23000 center 0.01
Goal4 20000 center 0.1
Goal5 15000 center 0.02
Goal6 20000 bottom 0.13
Goal7 22000 center 0.06
Goal8 20000 center 0.09
Goal9 22000 center 0.02
Goal10 20000 bottom 0.15
Goal11 23000 center 0.19
Goal12 23000 center 0.26
Goal13 20000 center 0.01
Goal14 21000 center 0.07
Goal15 22000 center 0.6
Goal16 23000 bottom 0.27
Goal17 21000 center 0.02


In [ ]:
# Showing F1 Score and Accuracy in all an by qry
import numpy as np
from sklearn.metrics import f1_score, accuracy_score

def show_statistics_valid_test(all_predited_list, all_tgt_list):
  by_qry_tgt_list = []
  by_qry_pred_list = []
  for i in range(17):
    by_qry_tgt = []
    for data in all_tgt_list:
      by_qry_tgt += data[i]
    by_qry_pred = []
    for data in all_predited_list:
      by_qry_pred += data[i]
    f1 = f1_score(by_qry_pred, by_qry_tgt)
    acc = accuracy_score(by_qry_pred, by_qry_tgt)
    print(f'Goal{i+1}, f1_score: {round(f1, 3)}, accuracy: {round(acc, 3)}')

    by_qry_tgt_list.append(by_qry_tgt)
    by_qry_pred_list.append(by_qry_pred)
  
  tgt_list_all = []
  pred_list_all = []
  for by_qry_tgt in by_qry_tgt_list:
    tgt_list_all += by_qry_tgt
  for by_qry_pred in by_qry_pred_list:
    pred_list_all += by_qry_pred
  
  f1_all = f1_score(pred_list_all, tgt_list_all)
  acc_all = accuracy_score(pred_list_all, tgt_list_all)
  print(f'Total, f1_score: {round(f1_all, 3)}, accuracy: {round(acc_all, 3)}')

In [ ]:
import torch
from glob import glob
import numpy as np

def valid_test_execute(params_list, vt_flag):
  dataset = torch.load(glob(work_dir + '/dataset.pt')[0])
  optim_list_all = []
  for i, step, align, _ in params_list:
    pt_optim_dicts_all = torch.load(glob(work_dir + '/results/' + str(i) + '/' + vt_flag + '.step' + str(step) + '.optim_dicts_all.pt')[0])
    optim_dicts_all = gen_optim_list(pt_optim_dicts_all, str(i), align)
    optim_list_by_qry = []
    for optim_dict in optim_dicts_all:
      optim_list_by_qry.append(torch.tensor([optim_dict[str(i)][j][0] for j in sorted(optim_dict[str(i)])]))
    optim_list_all.append(optim_list_by_qry)

  src_pred_list = []
  data_type = 'tgt_list_' + vt_flag
  for i, _, _, threshold in params_list:
    src_pred = predict_goals(optim_list_all[i-1], threshold=threshold)
    src_pred_list.append(src_pred)
  src_pred_list_t = np.array(src_pred_list).T.tolist()
  show_statistics_valid_test(src_pred_list_t, dataset[data_type])

In [ ]:
valid_test_execute(params_list, 'valid')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


Goal1, f1_score: 0.212, accuracy: 0.981
Goal2, f1_score: 0.505, accuracy: 0.992
Goal3, f1_score: 0.433, accuracy: 0.923
Goal4, f1_score: 0.529, accuracy: 0.983
Goal5, f1_score: 0.515, accuracy: 0.959
Goal6, f1_score: 0.446, accuracy: 0.99
Goal7, f1_score: 0.548, accuracy: 0.952
Goal8, f1_score: 0.515, accuracy: 0.924
Goal9, f1_score: 0.417, accuracy: 0.925
Goal10, f1_score: 0.403, accuracy: 0.974
Goal11, f1_score: 0.503, accuracy: 0.961
Goal12, f1_score: 0.567, accuracy: 0.948
Goal13, f1_score: 0.545, accuracy: 0.945
Goal14, f1_score: 0.547, accuracy: 0.976
Goal15, f1_score: 0.578, accuracy: 0.976
Goal16, f1_score: 0.447, accuracy: 0.978
Goal17, f1_score: 0.323, accuracy: 0.937
Total, f1_score: 0.485, accuracy: 0.96


In [ ]:
################ Test ################

In [ ]:
!cp $MODULE_PATH/bert_config_uncased_base_proposed.json ../bert_config_uncased_base.json

%env LOG_PATH=$work_dir/logs/proposed

for idx, steps, _, _ in params_list:
  %env RESULT_PATH=$work_dir/results/$idx/ 
  %env BERT_DATA_PATH=$source_dir/pt_data/$idx/
  %env MODEL_PATH=$work_dir/models/proposed$idx
  model_name = 'model_step_' + str(steps) + '.pt'
  %env MODEL_NAME=$model_name
  !python $MODULE_PATH/src/train.py -mode test -summarizer proposed_sigmoid -bert_data_path $BERT_DATA_PATH -visible_gpus 0  -gpu_ranks 0 -batch_size 512 -log_file $LOG_PATH  -result_path $RESULT_PATH -test_from $MODEL_PATH/$MODEL_NAME -block_trigram true

env: LOG_PATH=/content/drive/MyDrive/work_proposed_sigmoid/logs/proposed
env: RESULT_PATH=/content/drive/MyDrive/work_proposed_sigmoid/results/1/
env: BERT_DATA_PATH=/content/drive/MyDrive/work_proposed/pt_data/1/
env: MODEL_PATH=/content/drive/MyDrive/work_proposed_sigmoid/models/proposed1
env: MODEL_NAME=model_step_17000.pt
[2022-04-22 16:56:39,980 INFO] Loading checkpoint from /content/drive/MyDrive/work_proposed_sigmoid/models/proposed1/model_step_17000.pt
Namespace(accum_count=1, batch_size=512, bert_config_path='../bert_config_uncased_base.json', bert_data_path='/content/drive/MyDrive/work_proposed/pt_data/1/', beta1=0.9, beta2=0.999, block_trigram=True, dataset='', decay_method='', dropout=0.1, ff_size=2048, gpu_ranks=[0], heads=8, hidden_size=128, inter_layers=2, log_file='/content/drive/MyDrive/work_proposed_sigmoid/logs/proposed', lr=1, max_grad_norm=0, mode='test', model_path='../models/', optim='adam', param_init=0, param_init_glorot=True, recall_eval=False, report_every=1,

In [ ]:
# Showing Test Results
valid_test_execute(params_list, 'test')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


Goal1, f1_score: 0.177, accuracy: 0.984
Goal2, f1_score: 0.253, accuracy: 0.989
Goal3, f1_score: 0.364, accuracy: 0.919
Goal4, f1_score: 0.393, accuracy: 0.975
Goal5, f1_score: 0.421, accuracy: 0.951
Goal6, f1_score: 0.611, accuracy: 0.987
Goal7, f1_score: 0.455, accuracy: 0.945
Goal8, f1_score: 0.345, accuracy: 0.896
Goal9, f1_score: 0.367, accuracy: 0.917
Goal10, f1_score: 0.202, accuracy: 0.963
Goal11, f1_score: 0.3, accuracy: 0.927
Goal12, f1_score: 0.436, accuracy: 0.93
Goal13, f1_score: 0.542, accuracy: 0.931
Goal14, f1_score: 0.439, accuracy: 0.968
Goal15, f1_score: 0.499, accuracy: 0.977
Goal16, f1_score: 0.36, accuracy: 0.974
Goal17, f1_score: 0.197, accuracy: 0.934
Total, f1_score: 0.389, accuracy: 0.951
